In [1]:
import pandas as pd
import commons as me

knownLos = pd.read_csv('../data/clean/known_los.csv')
len(knownLos)

73

In [2]:
# Removing LOS where I could not pinpoint the observer or target location

knownLos = knownLos.dropna(subset=['from_latitude', 'from_longitude', 'to_latitude', 'to_longitude'])
knownLos['curvature'] = None

len(knownLos)

46

In [3]:
# Removing LOS where the reported distance does not match the actual distance based on identifications

def getKnownLosDistance(los):
    return me.geod.line_length([los.from_longitude, los.to_longitude], [los.from_latitude, los.to_latitude])

knownLos['distance_reported'] = knownLos.distance
knownLos['distance_true'] = round(knownLos.apply(getKnownLosDistance, axis=1)/1000, 3) # in km
knownLos = knownLos.query('abs(distance_true - distance) < 1')

len(knownLos)

34

In [4]:
# Finding the minimum light curvature needed for all known LOS to be possible

testCurvature = 7.7 # Max light curvature to test

while sum(knownLos.curvature.isna()) > 0 and testCurvature >= 1.0: # testCurvature = 1 gives the best LOS conditions

    print(f'Test Curvature: {round(testCurvature, 1)}')
    print(f'LOS Remaining: {sum(knownLos.curvature.isna())}\n')

    missingCurvatureMask = knownLos.curvature.isna()

    for i in knownLos.index[missingCurvatureMask]:
        los = knownLos.loc[i]
    
        observer = me.Summit(latitude=los.from_latitude,
                             longitude=los.from_longitude,
                             elevation=los.from_elevation)

        target = me.Summit(latitude=los.to_latitude,
                           longitude=los.to_longitude,
                           elevation=los.to_elevation)
    
        lineOfSight = me.LineOfSight(observer, target, lightCurvature=testCurvature)
        lineOfSight.processFullLineOfSight()
    
        if not lineOfSight.isObstructed():
            knownLos.at[i, 'curvature'] = round(testCurvature, 1)
    
    allCurvaturesFound = knownLos.curvature.notna().all()
    
    testCurvature -= 0.1

Test Curvature: 7.7
LOS Remaining: 34

Test Curvature: 7.6
LOS Remaining: 2

Test Curvature: 7.5
LOS Remaining: 2

Test Curvature: 7.4
LOS Remaining: 2

Test Curvature: 7.3
LOS Remaining: 2

Test Curvature: 7.2
LOS Remaining: 2

Test Curvature: 7.1
LOS Remaining: 2

Test Curvature: 7.0
LOS Remaining: 2

Test Curvature: 6.9
LOS Remaining: 2

Test Curvature: 6.8
LOS Remaining: 1

Test Curvature: 6.7
LOS Remaining: 1

Test Curvature: 6.6
LOS Remaining: 1

Test Curvature: 6.5
LOS Remaining: 1

Test Curvature: 6.4
LOS Remaining: 1



In [5]:
knownLos = knownLos[['from', 'from_area', 'from_elevation', 'from_latitude', 'from_longitude', 
                     'to', 'to_area', 'to_elevation', 'to_latitude', 'to_longitude', 
                     'distance_reported', 'distance_true', 'curvature']]

knownLos.head()

,from,from_area,from_elevation,from_latitude,from_longitude,to,to_area,to_elevation,to_latitude,to_longitude,distance_reported,distance_true,curvature
0,Finestrelles,Pirineos E (ES),2827,42.414475,2.133279,Pic Gaspard,Alpes (FR),3883,44.99811,6.33042,443,443.011,6.9
1,Bastiments,Pirineos E (ES),2881,42.426120,2.233078,Doigt de Dieu,Alpes (FR),3973,45.00382,6.31445,436,435.503,7.7
2,Bastiments,Pirineos E (ES),2881,42.426120,2.233078,Barre des Ecrins,Alpes (FR),4102,44.92215,6.35953,432,432.529,7.7
3,Canigou,Pirineos E (FR),2784,42.518976,2.456582,Pic Gaspard,Alpes (FR),3867,44.99811,6.33042,416,415.584,7.7
4,Canigou,Pirineos E (FR),2784,42.518976,2.456582,Barre des Ecrins,Alpes (FR),4102,44.92215,6.35953,412,411.961,7.7


In [6]:
knownLos.to_csv('../data/clean/known_los_with_curvature.csv', index=False)